In [ ]:
import pandas as pd
from itertools import combinations

# Correct file path assignment
file_path = "/Users/joenockels/douglass-kg/douglass_gpe.csv"
entities = pd.read_excel(file_path, sheet_name="douglass_gpe")

# Load sheets
# diary_entries = pd.read_excel(file_path, sheet_name="HP_diary_entries")
# entities = pd.read_excel(file_path, sheet_name="HP_nodes")
# mentions = pd.read_excel(file_path, sheet_name="HP_mentions")

In [ ]:
mentions = pd.read_excel(
    file_path,
    sheet_name="HP_mentions",
)

print(mentions.head())
print(mentions.columns)

In [ ]:
mentions_unique = mentions.drop_duplicates(subset=["entry_id", "entity_id"])

In [ ]:
mentions_unique = mentions_unique.dropna(subset=["entity_id", "entry_id"])
mentions_unique["entity_id"] = mentions_unique["entity_id"].astype(str)
mentions_unique["entry_id"] = mentions_unique["entry_id"].astype(str)

In [ ]:
from itertools import combinations
import pandas as pd

# edges list
edges = []

# Group mentions by diary entry
for entry_id, group in mentions_unique.groupby("entry_id"):
    
    # get unique entities in this entry
    entities_in_entry = sorted(group["entity_id"].unique())
    
    # grab the date of this entry
    date = group["standard_date"].iloc[0]
    
    # generate all pairwise combinations
    for source, target in combinations(entities_in_entry, 2):
        edges.append({
            "source": source,
            "target": target,
            "date": date
        })

# create dataframe
edges_df = pd.DataFrame(edges)

# quick sanity check
print(edges_df.head())
print(f"Total edges: {len(edges_df)}")

In [ ]:
dynamic_edges = (
    edges_df
    .groupby(["source", "target"])
    .agg(
        weight=("date", "count"),
        start=("date", "min"),
        end=("date", "max")
    )
    .reset_index()
)

print(dynamic_edges.head())
print(len(dynamic_edges))

In [ ]:
dynamic_edges["start"] = pd.to_datetime(dynamic_edges["start"])
dynamic_edges["end"] = pd.to_datetime(dynamic_edges["end"])

In [ ]:
dynamic_edges.to_csv("HP_edges.csv", index=False)
entities.to_csv("HP_nodes.csv", index=False)

In [4]:
import pandas as pd
from itertools import combinations

# Load your data
person_place_df = pd.read_csv("/Users/joenockels/douglass-kg/GPE - Person <-> Place.csv")

# Map: place -> set of all people across all timesets
place_people_total = person_place_df.groupby('Id')['Person'].apply(set).to_dict()

edges_list = []

# Generate all place pairs
for place1, place2 in combinations(place_people_total.keys(), 2):
    shared_people = place_people_total[place1] & place_people_total[place2]
    weight = len(shared_people)
    if weight > 0:
        edges_list.append({
            "Source": place1,
            "Target": place2,
            "Weight": weight
        })

edges_df = pd.DataFrame(edges_list)
edges_df.to_csv("place_edges_aggregated.csv", index=False)
print(edges_df)

    Source  Target  Weight
0        2       3       1
1        2       8       1
2        2       9       1
3        2      11       1
4        2      12       1
5        2      16       1
6        2      22       1
7        2      23       1
8        2      28       1
9        2      30       1
10       3       8       1
11       3       9       1
12       3      11       1
13       3      12       1
14       3      16       1
15       3      22       1
16       3      23       1
17       3      28       1
18       3      30       1
19       8       9       1
20       8      11       1
21       8      12       1
22       8      16       1
23       8      22       1
24       8      23       1
25       8      28       1
26       8      30       1
27       9      11       1
28       9      12       1
29       9      16       1
30       9      22       1
31       9      23       1
32       9      28       1
33       9      30       1
34      11      12       1
35      11      16       1
3